# Notebook to train number recognition model based on MNIST dataset.

Accomplished results on training set : accuracy: **0.9974** per 30 epochs

Results on testing set : **0.994**

CNN Based on **LeNet-5** architecture.

LeNet-5 GradientBased Learning Applied to Document Recognition (Yann LeCun Leon Bottou Yoshua Bengio and Patrick Haffner) (http://vision.stanford.edu/cs598_spring07/papers/Lecun98.pdf)

## Importing all required packages

Using tensorflow and keras library for the model itself

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Activation, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import math

### LeNet-5 Modified Architecture

ConvNet --> **ConvNet** --> **BatchNorm** --> Pool --> **(Dropout)** --> ConvNet --> **ConvNet** --> **BatchNorm** --> Pool --> **(Dropout)** --> (Flatten) --> **FullyConnected** --> **BatchNorm** --> FullyConnected --> **BatchNorm** --> FullyConnected --> **BatchNorm** --> **(Dropout)** --> Softmax 

### Implementation of modified architecture for training a model

In [ ]:
def LeNet5(input_shape = (32, 32, 1), classes = 10):

    model = Sequential([
        
    Conv2D(filters = 32, kernel_size = 5, strides = 1, activation = 'relu', input_shape = (32,32,1), kernel_regularizer=l2(0.0005), name = 'convolution_1'),
    
    Conv2D(filters = 32, kernel_size = 5, strides = 1, name = 'convolution_2', use_bias=False),
      
    BatchNormalization(name = 'batchnorm_1'),
         
    Activation("relu"),

    MaxPooling2D(pool_size = 2, strides = 2, name = 'max_pool_1'),

    Dropout(0.25, name = 'dropout_1'),

    Conv2D(filters = 64, kernel_size = 3, strides = 1, activation = 'relu', kernel_regularizer=l2(0.0005), name = 'convolution_3'),
        
    Conv2D(filters = 64, kernel_size = 3, strides = 1, name = 'convolution_4', use_bias=False),
        
    BatchNormalization(name = 'batchnorm_2'),
        
    Activation("relu"),

    MaxPooling2D(pool_size = 2, strides = 2, name = 'max_pool_2'),

    Dropout(0.25, name = 'dropout_2'),

    Flatten(name = 'flatten'),
        
    Dense(units = 256, name = 'fully_connected_1', use_bias=False),
        
    BatchNormalization(name = 'batchnorm_3'),
    
    Activation("relu"),
        
    Dense(units = 128, name = 'fully_connected_2', use_bias=False),
        
    BatchNormalization(name = 'batchnorm_4'),
        
    Activation("relu"),
        
    Dense(units = 84, name = 'fully_connected_3', use_bias=False),
        
    BatchNormalization(name = 'batchnorm_5'),
        
    Activation("relu"),

    Dropout(0.25, name = 'dropout_3'),

    Dense(units = 10, activation = 'softmax', name = 'output')
        
    ])
    
    model._name = 'LeNet5'

    return model

In [ ]:
LeNet5 = LeNet5(input_shape = (32, 32, 1), classes = 10)

LeNet5.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Kaggle data processing

MNIST Dataset used (https://www.kaggle.com/competitions/digit-recognizer/data)

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

Y = train[['label']]
X = train.drop(train.columns[[0]], axis=1)

X = X.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

cross_validation_size = int(len(X)*0.05)

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = cross_validation_size)

X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

X_train = np.pad(X_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_val = np.pad(X_val, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_test = np.pad(X_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')

mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)
X_train = (X_train - mean_px)/(std_px)

mean_px = X_val.mean().astype(np.float32)
std_px = X_val.std().astype(np.float32)
X_val = (X_val - mean_px)/(std_px)

mean_px = X_test.mean().astype(np.float32)
std_px = X_test.std().astype(np.float32)
X_test = (X_test - mean_px)/(std_px)

Y_train = to_categorical(Y_train, num_classes = 10)
Y_val = to_categorical(Y_val, num_classes = 10)

datagen = ImageDataGenerator(
        featurewise_center = False,
        samplewise_center = False,
        featurewise_std_normalization = False,
        samplewise_std_normalization = False,
        zca_whitening = False,
        rotation_range = 10,
        zoom_range = 0.1,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        horizontal_flip = False,
        vertical_flip = False) 

datagen.fit(X_train)

## Learning rate configuration

In [ ]:
variable_learning_rate = ReduceLROnPlateau(monitor='val_loss', factor = 0.2, patience = 2)

## Training the model

In [ ]:
history = LeNet5.fit(X_train, Y_train, epochs = 30, batch_size = 64, callbacks = [variable_learning_rate], validation_data = (X_val,Y_val))

# References 

https://github.com/guptajay/Kaggle-Digit-Recognizer
https://www.kaggle.com/code/yassineghouzam/introduction-to-cnn-keras-0-997-top-6/notebook#4.-Evaluate-the-model